In [36]:
import pandas as pd
import numpy as np
import re
import os
import warnings
import time
from time import time, sleep



# 경고창 무시하는 코드
warnings.filterwarnings('ignore')

#clean text 사용할 함수
def cleanText(readData):
    # 괄호랑 그 내용 없애기
    text = re.sub(r'\([^)]*\)', '', readData)
    return text
def cleanText2(readData):    # 특수문자 없애기
    text = re.sub('[-=+,#/\?:;^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》❤]', '', readData)
    return text


path = str(input('file path: '))
path2 = str(input('orders 파일을 저장할 경로'))
lst = os.listdir(path)
start = time()
for i in lst:     
    df = pd.read_csv(path + f'{i}', encoding='cp949')
    
    orders = df[['주문번호', # 주문번호
    '품목별 주문번호', # 주문상품명
    '주문자 ID', # 고객ID
    '주문상품코드', # 주문상품코드 (카테고리)
    '옵션별상품코드', # 상품코드
    '주문상품명(기간할인 제목+버전)', # 주문상품명
    '구매횟수', # 이전 구매 횟수
    '주문품목 수량', # 수량
    '주문품목 결제금액', # 주문금액
    '주문품목 할인금액', # 주문품목 할인금액
    '주문일시', # 주문일시
    '주문경로', # 주문경로
    '주문검색어', # 주문검색어
    '수령인 우편번호(XXXXXX)', # 우편번호
    '수령인 주소1', #주소
    '배송구분', # 배송구분
    '네이버 페이 주문번호', # 네이버 페이 주문번호
    '결제금액(주문전체)', # 결제금액
    '할인 상세내역(주문전체)', # 할인상세
    '사용한 적립금', #사용적립금
    '사용한 예치금', #사용한 예치금
    '결제방법', # 결제방법
    '결제상태', # 결제상태
    '주문상태(품목별)', #품목별주문상태
    '입금확인일(결제일)', # 결제일
    '송장입력일', # 송장입력일
    '배송중처리일', #배송중처리일
    '배송완료일']] #배송완료일

    # 컬럼 rename
    orders.rename(columns={'주문상품명(기간할인 제목+버전)':'주문상품명', 
                                 '주문품목 수량':'수량','주문품목 결제금액':'주문금액', 
                                 '결제금액(주문전체)':'결제금액', 
                                 '할인 상세내역(주문전체)':'할인상세', 
                                 '사용한 적립금':'사용적립금',
                                 '주문상태(품목별)':'품목별주문상태', 
                                 '입금확인일(결제일)':'결제일',
                                '수령인 우편번호(XXXXXX)':'우편번호',
                                  '수령인 주소1':'주소', '옵션별상품코드':'상품코드',
                          }, inplace=True)



    # 주문 상품 코드 세자리로 발라내기
    orders['주문상품코드'] = orders['주문상품코드'].str.replace('.', '').str.slice(stop=3)

    ### orders 주문상품 999인 것 (사은품) 제거
    orders = orders[orders['주문상품명'] != 999].reset_index(drop=True)


    ## 실제 orders 데이터에 적용

    orders['주문상품명'] = orders['주문상품명'].astype('str')

    # 괄호 안의 내용 삭제하기
    orders['주문상품명'] = orders['주문상품명'].str.replace('[', '(').str.replace(']', ')')
    orders['주문상품명'] = orders['주문상품명'].apply(cleanText)
    # 특수문자 및 공백 제거
    orders['주문상품명'] = orders['주문상품명'].apply(cleanText2)
    orders['주문상품명'] = orders['주문상품명'].str.strip()

    ## 네이버 페이 주문번호 컬럼 -> 네이버페이주문(0/1)
    # 네이버페이 컬럼을 일단 1 값으로 채워줌
    orders['네이버페이'] = 1

    # 그 중 네이버 페이 주문번호가 null인 값은 0을 넣어줌
    orders['네이버페이'][orders['네이버 페이 주문번호'].isnull()] = 0
    orders.drop('네이버 페이 주문번호', axis=1, inplace=True)



    ## 사용한 예치금 1,0으로 만들기
    # 예치금 사용 컬럼을 일단 1 값으로 채워줌
    orders['예치금사용'] = 1

    # 그 중 예치금 사용이 null인 값은 0을 넣어줌
    orders['예치금사용'][orders['사용한 예치금'] == 0] = 0
    orders[['사용한 예치금', '예치금사용']]
    orders.drop('사용한 예치금', axis=1, inplace=True)

    ### 결제 시간대 추가

    orders['결제시간대'] = orders['결제일'].str.split(' ').str[1].str.split(':').str[0] + '시대'


    ## 요일 추가
    import datetime

    orders['요일'] = orders['결제일'].astype('datetime64').dt.weekday
    # 0 = Monday, 6= Sunday

    orders['요일'][orders['요일'] == 0] = '월'
    orders['요일'][orders['요일'] == 1] = '화'
    orders['요일'][orders['요일'] == 2] = '수'
    orders['요일'][orders['요일'] == 3] = '목'
    orders['요일'][orders['요일'] == 4] = '금'
    orders['요일'][orders['요일'] == 5] = '토'
    orders['요일'][orders['요일'] == 6] = '일'
    
    orders = orders.rename(columns={
                    '주문자 ID': 'id',
                    '우편번호':'postal_code',
                    '주소':'address',
                    '주문번호':'order_id',
                    '품목별 주문번호':'order_detail',
                    '주문상품코드':'category',
                    '상품코드':'product_id',
                    '주문상품명':'product_name',
                    '수량':'qty',
                    '주문금액':'amount',
                    '주문품목 할인금액':'discount',
                    '주문일시':'order_date',
                    '주문경로':'order_path',
                    '주문검색어':'search',
                    '배송구분':'delivery',
                    '결제금액':'total_price',
                    '할인상세':'discout_detail',
                    '사용적립금':'point',
                    '사용한 예치금':'deposit',
                    '결제방법':'payment_type',
                    '결제상태':'payment_status',
                    '품목별주문상태':'order_status',
                    '결제일':'payment_date',
                    '송장입력일':'order_invoicedate',
                    '배송중처리일':'order_delivered_date',
                    '배송완료일':'order_delivered_customer_date',
                    '네이버페이':'naverpay',
                    '예치금사용':'deposit_use',
                    '결제시간대':'timerange',
                    '요일':'weekday',
                    '구매횟수':'cnt_orders'})
    
    ## 1102 postal code 6자리로 맞추는 코드 추가
    orders['postal_code'] = orders['postal_code'].astype('str').apply(cleanText2).str.ljust(6,'0')
    
    try:        
        old = pd.read_csv(path2 + 'orders.csv')
        new = pd.concat([orders, old])
        print('concat 완료')
        new = new.drop_duplicates().reset_index(drop=True) # 중복 제거
        new.to_csv(path2 + 'orders.csv', index = False)
        # 다운로드
    
    except FileNotFoundError:        
        print('파일을 찾을 수 없습니다.')
        # 1102 orders 초기 저장 시 중복 제거하는 코드 추가
        orders = orders.drop_duplicates()
        orders.to_csv(path2 + 'orders.csv', index = False)
        
    print(f'{i} 저장 완료')


end = time()
print('소요시간(분):', round((end - start) / 60))


file path: C:\Users\monam\Desktop\dataitgirls 6\데모데이\dataset\raw\
orders 파일을 저장할 경로C:\Users\monam\Desktop\dataitgirls 6\데모데이\dataset\preprocessed\
파일을 찾을 수 없습니다.
dwn_order_banulfren_1_20200131.csv 저장 완료
concat 완료
dwn_order_banulfren_1_20200231.csv 저장 완료
concat 완료
dwn_order_banulfren_1_20200331.csv 저장 완료
concat 완료
dwn_order_banulfren_1_20200431.csv 저장 완료
concat 완료
dwn_order_banulfren_1_20200531.csv 저장 완료
concat 완료
dwn_order_banulfren_1_20200631.csv 저장 완료
concat 완료
dwn_order_banulfren_1_20200731.csv 저장 완료
concat 완료
dwn_order_banulfren_1_20200831.csv 저장 완료
concat 완료
dwn_order_banulfren_1_20200931.csv 저장 완료
concat 완료
dwn_order_banulfren_1_20201031.csv 저장 완료
concat 완료
dwn_order_banulfren_1_20201131.csv 저장 완료
concat 완료
dwn_order_banulfren_1_20201231.csv 저장 완료
concat 완료
dwn_order_banulfren_1_20210131.csv 저장 완료
concat 완료
dwn_order_banulfren_1_20210231.csv 저장 완료
concat 완료
dwn_order_banulfren_1_20210331.csv 저장 완료
concat 완료
dwn_order_banulfren_1_20210431.csv 저장 완료
concat 완료
dwn_order_banulfren_1_

In [2]:
df.columns

Index(['주문번호', '품목별 주문번호', '배송번호', '주문상품코드', '주문상품명', '주문상품명(옵션포함)',
       '주문상품명(옵션/수량포함)', '주문상품명(기간할인 제목+버전)', '매입처상품명(옵션포함)', '상품옵션',
       '상품옵션(옵션가격미노출)', '상품옵션(옵션타이틀미노출)', '옵션별상품코드', '주문품목 수량', '주문품목 단가',
       '주문품목 결제금액', '주문품목 할인금액', '주문일시', '주문경로', '주문검색어', '주문자 ID', '구매횟수',
       '주문자 연령', '주문자 생일/성별', '주문자 국가명', '회원추천인', '주문자 그룹', '고객보유적립금',
       '수령인 우편번호(XXXXXX)', '수령인 우편번호(XXX-XXX)', '수령인 주소1', '배송구분', '수령인 국가명',
       '수령인 국가코드', '수령인 국가 전화번호', '송장번호', '배송료(+특수지역배송비 포함)', '환불배송료',
       'Unnamed: 38', '물류메시지', '물류메시지(한줄로)', 'Unnamed: 41', 'Unnamed: 42',
       '배송비종류', '네이버 페이 주문번호', '네이버 페이 상품별 주문번호', '네이버 페이 주문메시지',
       '네이버 페이 옵션추가정보', '결제금액(주문전체)', '할인 상세내역(주문전체)', '쿠폰할인내역(품목별)',
       '회원등급할인내역(품목별)', '결제금액(품목별)', '사용한 적립금', '사용한 예치금', '사용한 네이버 포인트',
       '결제방법', '결제상태', '주문상태(품목별)', '입금확인일(결제일)', '송장입력일', '배송중처리일', '배송완료일'],
      dtype='object')

In [37]:
namyi=pd.read_csv(path2 + r'\orders.csv')
namyi

,order_id,order_detail,id,category,product_id,product_name,cnt_orders,qty,amount,discount,...,payment_status,order_status,payment_date,order_invoicedate,order_delivered_date,order_delivered_customer_date,naverpay,deposit_use,timerange,weekday
0,20221024150449-97654811689,20221024150449-97654811689_[1],ka@5629ed2b,117,M3446110-00000000012,테디 울,0.0,1.0,6500.0,0,...,입금완료,상품준비,2022-10-24 15:05,NaN,NaN,NaN,0,0,15시대,월
1,20221024150449-97654811689,20221024150449-97654811689_[2],ka@5629ed2b,117,M3446110-00000000013,테디 울,0.0,2.0,13000.0,0,...,입금완료,상품준비,2022-10-24 15:05,NaN,NaN,NaN,0,0,15시대,월
2,20221024150449-97654811689,20221024150449-97654811689_[3],ka@5629ed2b,12E,M10338268-0000000001,마인드풀 13cm 조립식 실속세트,0.0,1.0,110000.0,0,...,입금완료,상품준비,2022-10-24 15:05,NaN,NaN,NaN,0,0,15시대,월
3,20221024150429-25233284822,20221024150429-25233284822_[1],sjyoung71,117,M3469103-00000000001,새틴 알파카 모헤어,6.0,1.0,10000.0,0,...,입금완료,배송준비,2022-10-24 15:04,NaN,NaN,NaN,0,0,15시대,월
4,20221024150429-25233284822,20221024150429-25233284822_[2],sjyoung71,117,M10338920-0000000002,멀버리 실크 모헤어,6.0,1.0,13000.0,0,...,입금완료,배송준비,2022-10-24 15:04,NaN,NaN,NaN,0,0,15시대,월
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1889168,20200101002929-06154896323,20200101002929-06154896323_[1],dadav,117,M2145613-00000000002,윌로우,30.0,4.0,60000.0,0,...,입금완료,거래완료,2020-01-01 0:30,2020-01-02 13:53,2020-01-02 13:53,2020-01-03 10:30,0,0,0시대,수
1889169,20200101002929-06154896323,20200101002929-06154896323_[2],dadav,118,M3379134-00000000001,윌로우 에디션 선데이 가디건,30.0,1.0,114000.0,0,...,입금완료,거래완료,2020-01-01 0:30,2020-01-02 13:53,2020-01-02 13:53,2020-01-03 10:30,0,0,0시대,수
1889170,20200101002929-06154896323,20200101002929-06154896323_[3],dadav,999,M0-00000000000000000,스노우 플레이크 2볼,30.0,1.0,0.0,0,...,입금완료,거래완료,2020-01-01 0:30,2020-01-02 13:53,2020-01-02 13:53,2020-01-03 10:30,0,0,0시대,수
1889171,20200101002310-90431230989,20200101002310-90431230989_[1],cdiwk297,117,M10272-0000000000014,라피도,3.0,2.0,14000.0,0,...,비뱅크자동입금완료,거래완료,2020-01-01 1:04,2020-01-02 11:59,2020-01-02 11:59,2020-01-03 12:50,0,0,1시대,수


In [38]:
idx=namyi[namyi['discount'] == '무료'].index

In [39]:
idx

Int64Index([1220652], dtype='int64')

In [40]:
namyi = namyi.drop(idx).reset_index(drop=True)

In [41]:
namyi.order_date.unique()

array(['2022-10-24 15:04', '2022-10-24 15:03', '2022-10-24 15:02', ...,
       '2020-01-01 0:29', '2020-01-01 0:23', '2020-01-01 0:05'],
      dtype=object)

In [43]:
namyi[namyi['discount'] == '무료']

,order_id,order_detail,id,category,product_id,product_name,cnt_orders,qty,amount,discount,...,payment_status,order_status,payment_date,order_invoicedate,order_delivered_date,order_delivered_customer_date,naverpay,deposit_use,timerange,weekday


In [44]:
namyi.to_csv(path2 + 'orders.csv', index = False)

In [33]:
#postal code 수정하는 코드
orders['postal_code'].astype('str').apply(cleanText2).str.ljust(6,'0')

0        275020
1        275020
2        275020
3        626200
4        626200
          ...  
45622    507000
45623    507000
45624    507000
45625    507000
45626    507000
Name: postal_code, Length: 45627, dtype: object

In [35]:
orders['order_date'].isnull().sum()

0

In [55]:
len(namyi['postal_code']) / 6

314862.0

In [73]:
namyi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889172 entries, 0 to 1889171
Data columns (total 30 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   order_id                       object 
 1   order_detail                   object 
 2   id                             object 
 3   category                       object 
 4   product_id                     object 
 5   product_name                   object 
 6   cnt_orders                     float64
 7   qty                            float64
 8   amount                         float64
 9   discount                       object 
 10  order_date                     object 
 11  order_path                     object 
 12  search                         object 
 13  postal_code                    object 
 14  address                        object 
 15  delivery                       object 
 16  total_price                    float64
 17  discout_detail                 object 
 18  po

In [74]:
namyi.order_date

0          2022-10-24 15:04
1          2022-10-24 15:04
2          2022-10-24 15:04
3          2022-10-24 15:04
4          2022-10-24 15:04
                 ...       
1889167     2020-01-01 0:29
1889168     2020-01-01 0:29
1889169     2020-01-01 0:29
1889170     2020-01-01 0:23
1889171     2020-01-01 0:05
Name: order_date, Length: 1889172, dtype: object

In [69]:
namyi.columns

Index(['order_id', 'order_detail', 'id', 'category', 'product_id',
       'product_name', 'cnt_orders', 'qty', 'amount', 'discount', 'order_date',
       'order_path', 'search', 'postal_code', 'address', 'delivery',
       'total_price', 'discout_detail', 'point', 'payment_type',
       'payment_status', 'order_status', 'payment_date', 'order_invoicedate',
       'order_delivered_date', 'order_delivered_customer_date', 'naverpay',
       'deposit_use', 'timerange', 'weekday'],
      dtype='object')

# customers

In [2]:
import pandas as pd
import numpy as np
import re
import os
import fnmatch
import warnings
import time
from time import time, sleep



# 경고창 무시하는 코드
warnings.filterwarnings('ignore')


# 미리 입력받아야 하는 함수
path = str(input('raw file path: '))
path2 = str(input('customers 파일을 저장할 경로'))
lst = os.listdir(path)
start = time()

for i in lst:     
    df = pd.read_csv(path + f'{i}', encoding='cp949')
    
    dfc = df[['주문자 ID', '구매횟수', '주문자 연령', '주문자 생일/성별', '주문자 그룹', '수령인 우편번호(XXXXXX)', '수령인 주소1', '입금확인일(결제일)']]
    
    dfc = dfc.rename(columns = {'주문자 ID': '회원ID', '주문자 그룹' : '회원등급', '주문자 연령':'나이', '수령인 우편번호(XXXXXX)' : '우편번호', '수령인 주소1': '주소', '입금확인일(결제일)' : '결제일'})
    ## 5) 칼럼 순서 변경

    dfc = dfc[['회원ID', '구매횟수', '주문자 생일/성별', '나이','회원등급', '우편번호', '주소', '결제일']]
    
    # 결제일 컬럼 datetime으로 변환
    dfc['결제일'] = dfc['결제일'].astype('datetime64')
    
    # ['나이'] 컬럼에 2020년에는 +2, 2021년에는 +1
    dfc['나이'][dfc['결제일'].dt.year ==  2020] = dfc['나이'] +2
    dfc['나이'][dfc['결제일'].dt.year ==  2021] = dfc['나이'] +1
    
    ## 4) 결제일 시간 버리고 날짜만

    dfc['결제일'] = dfc['결제일'].astype('str')

    dfc['결제일'] = dfc['결제일'].str.split(' ').str[0]



    ## 2) '주문자 생일/성별' 칼럼 지우고 '성별'만 남기기

    dfc['성별'] = dfc['주문자 생일/성별'].str.split('/').str[1]

    dfc = dfc.drop(['주문자 생일/성별'], axis = 'columns')


    ## 3) 연령대 칼럼 만들기(123인 outliers를 null값으로 변경)

    dfc['나이'] = dfc['나이'].replace(dfc['나이'].max(), np.NaN)

    def get_age_group(age):
        group = ''
        if age <= 18 : group = '13-18'
        elif age <= 24 : group = '20-24'
        elif age <= 29 : group = '25-29'
        elif age <= 34 : group = '30-34'
        elif age <= 39 : group = '35-39'
        elif age <= 44 : group = '40-44'
        elif age <= 49 : group = '45-49'
        elif age <= 54 : group = '50-54'
        elif age <= 59 : group = '55-59'
        elif age <= 64 : group = '60-64'
        elif age <= 69 : group = '65-69'
        elif age <= 74 : group = '70-74'
        elif age <= 79 : group = '75-79'
        elif age <= 84 : group = '80-84'
        elif age <= 89 : group = '85-89'
        else : group = 'NaN'
        return group

    dfc['나이대'] = dfc['나이'].apply(lambda x : get_age_group(x))


    try:        
        
        old = pd.read_csv(path2 + 'customers.csv')
        new = pd.concat([dfc, old])
        print('concat 완료')
        new = new.reset_index(drop=True) # 인덱스 정리
        new.to_csv(path2 + 'customers.csv', index = False)
        # 다운로드
    except FileNotFoundError:        
        print('파일을 찾을 수 없습니다.')
        dfc.to_csv(path2 + 'customers.csv', index = False)
        

    print(f'{i} 저장 완료')
    
end = time()
print('소요시간(분):', round((end - start) / 60), 2)

raw file path: C:\Users\monam\Desktop\dataitgirls 6\데모데이\dataset\raw\
customers 파일을 저장할 경로C:\Users\monam\Desktop\dataitgirls 6\데모데이\dataset\preprocessed\
파일을 찾을 수 없습니다.
dwn_order_banulfren_1_20200131.csv 저장 완료
concat 완료
dwn_order_banulfren_1_20200231.csv 저장 완료
concat 완료
dwn_order_banulfren_1_20200331.csv 저장 완료
concat 완료
dwn_order_banulfren_1_20200431.csv 저장 완료
concat 완료
dwn_order_banulfren_1_20200531.csv 저장 완료
concat 완료
dwn_order_banulfren_1_20200631.csv 저장 완료
concat 완료
dwn_order_banulfren_1_20200731.csv 저장 완료
concat 완료
dwn_order_banulfren_1_20200831.csv 저장 완료
concat 완료
dwn_order_banulfren_1_20200931.csv 저장 완료
concat 완료
dwn_order_banulfren_1_20201031.csv 저장 완료
concat 완료
dwn_order_banulfren_1_20201131.csv 저장 완료
concat 완료
dwn_order_banulfren_1_20201231.csv 저장 완료
concat 완료
dwn_order_banulfren_1_20210131.csv 저장 완료
concat 완료
dwn_order_banulfren_1_20210231.csv 저장 완료
concat 완료
dwn_order_banulfren_1_20210331.csv 저장 완료
concat 완료
dwn_order_banulfren_1_20210431.csv 저장 완료
concat 완료
dwn_order_banul

In [48]:
jiwon = pd.read_csv(path2 + r'\customers.csv')
jiwon

,id,cnt_orders,age,membership,postal_code,address,last_order_date,gender,age.1
0,kyungahy,0.0,53.0,일반회원,4526.0,서울 중구 남대문로1길 34,2022-10-24,여자,50-54
1,ka@35607f9,0.0,31.0,일반회원,13562.0,경기 성남시 분당구 정자일로 100,2022-10-24,남자,30-34
2,ka@ac892714,0.0,36.0,일반회원,4137.0,서울 마포구 백범로21길 14,2022-10-24,여자,35-39
3,yena6592,0.0,59.0,일반회원,12233.0,경기 남양주시 사릉로34번길 21,2022-10-24,여자,55-59
4,ka@6355ccf,0.0,18.0,일반회원,49229.0,부산 서구 고운들로 181,2022-10-24,여자,13-18
...,...,...,...,...,...,...,...,...,...
112519,welovejy,1.0,52.0,일반회원,402809.0,인천 미추홀구 주안서로33번길 51,2020-01-01,여자,50-54
112520,ineco7274,3.0,32.0,일반회원,435010.0,경기 군포시 용호1로 57,2020-01-01,여자,30-34
112521,mina2929,2.0,40.0,일반회원,660796.0,경남 진주시 금산면 금산로 123,2020-01-01,여자,40-44
112522,micofine,3.0,38.0,일반회원,742961.0,경북 상주시 사벌면 화달2길 26-2,2020-01-01,여자,35-39


In [ ]:
jiwon[jiwon['Unnamed: 8'].notnull() == True] # 장난해..?

In [4]:
## 6) 정보가 2개 이상인 사람들, 결제일 기준 정렬 후 최근 결제일 정보만 남기기

cnt=jiwon.groupby('회원ID').count()
cnt=cnt[cnt['구매횟수'] >= 2].reset_index()
lst = list(cnt.index)

# dup : 회원ID 정보가 2개 이상인 행들만 모아놓은 df
dup = jiwon.merge(right=cnt['회원ID'], how='inner').drop_duplicates()

# oneID : dup에서 결제일이 최근인 ID정보만 남김
oneID = jiwon.merge(right=cnt['회원ID'], how='inner').sort_values(by='결제일', ascending=False).drop_duplicates('회원ID', keep='first')
oneID

### 1. df : 전처리 완료한 dfc에서 dup(중복되는 id 칼럼들만 있는 df) 제거하기

customers = pd.merge(
    dup, jiwon, how='outer', indicator = True
).query(
    '_merge == "right_only"'
    ).drop(columns=['_merge']
    )

### 2. dfc에서 dup 뺀 df에 oneID concat하기

customers = pd.concat([customers, oneID])


In [20]:
customers=customers.dropna(subset=['회원ID']).reset_index(drop=True)

In [21]:
tt = pd.DataFrame(customers.groupby('회원ID').회원ID.nunique())

In [24]:
tt[tt['회원ID'] >= 2]

,회원ID
회원ID,


In [46]:
customers.rename(columns={'회원ID':'id',
                         '구매횟수':'cnt_orders',
                         '나이':'age',
                         '회원등급':'membership',
                         '우편번호':'postal_code',
                         '주소':'address',
                         '결제일':'last_order_date',
                         '성별':'gender',
                         '나이대':'ageband'}, inplace=True)

In [36]:
customers = customers.drop_duplicates(subset=['id']).reset_index(drop=True)

In [57]:
customers.to_csv(path2 + r'\customers.csv', index=False)

In [55]:
col=['id', 'cnt_orders', 'age', 'membership', 'postal_code', 'address',
       'last_order_date', 'gender', 'ageband']
customers.columns = col

In [56]:
customers

,id,cnt_orders,age,membership,postal_code,address,last_order_date,gender,ageband
0,kyungahy,0.0,53.0,일반회원,4526,서울 중구 남대문로1길 34,2022-10-24,여자,50-54
1,ka@35607f9,0.0,31.0,일반회원,13562,경기 성남시 분당구 정자일로 100,2022-10-24,남자,30-34
2,ka@ac892714,0.0,36.0,일반회원,4137,서울 마포구 백범로21길 14,2022-10-24,여자,35-39
3,yena6592,0.0,59.0,일반회원,12233,경기 남양주시 사릉로34번길 21,2022-10-24,여자,55-59
4,ka@6355ccf,0.0,18.0,일반회원,49229,부산 서구 고운들로 181,2022-10-24,여자,13-18
...,...,...,...,...,...,...,...,...,...
112519,welovejy,1.0,52.0,일반회원,402809,인천 미추홀구 주안서로33번길 51,2020-01-01,여자,50-54
112520,ineco7274,3.0,32.0,일반회원,435010,경기 군포시 용호1로 57,2020-01-01,여자,30-34
112521,mina2929,2.0,40.0,일반회원,660796,경남 진주시 금산면 금산로 123,2020-01-01,여자,40-44
112522,micofine,3.0,38.0,일반회원,742961,경북 상주시 사벌면 화달2길 26-2,2020-01-01,여자,35-39


In [43]:
customers.cnt_orders.sort_values(ascending=False)

11470    420.0
58560    339.0
10908    277.0
19743    245.0
29202    238.0
         ...  
7601       NaN
7637       NaN
7688       NaN
7706       NaN
7762       NaN
Name: cnt_orders, Length: 112524, dtype: float64

In [58]:
customers['last_order_date'].sort_values()

112523    2020-01-01
112500    2020-01-01
112499    2020-01-01
112498    2020-01-01
112497    2020-01-01
             ...    
11575            NaT
11574            NaT
11573            NaT
11571            NaT
9976             NaT
Name: last_order_date, Length: 112524, dtype: object